In [1]:
import ipynb.fs.full.provenance as pr
import ipynb.fs.full.provenance_lib as pr_lib
import pandas as pd

from IPython.display import Image

# Specify where to save the processed files as savepath
savepath = 'results/COMPAScleanup_prov/'

In [2]:
# Files get loaded from fairCorrect github repository
url = 'https://raw.githubusercontent.com/vladoxNCL/fairCorrect/master/Datasets/'
df = pd.read_csv(url + 'compas-scores-two-years.csv', header=0)

m, n= df.shape
print("IN - Rows: ", m, " Features: ",n)

# Create a new provenance document and input entities 
#p = pr.Provenance(df, savepath)
p = pr_lib.Provenance(df, savepath)

IN - Rows:  7214  Features:  53


In [3]:
# select relevant columns
df =df[['c_charge_degree', 'race', 'two_year_recid', 'c_jail_in', 'c_jail_out']]

d = p.get_prov_feature_selection(df)

get_prov_feature_selection function took 8681.953 ms


In [4]:
# Remove missing values
df = df.dropna()

d = p.get_prov_dim_reduction(df)

get_prov_dim_reduction function took 271.327 ms


In [5]:
# Make race binary
df.race = [0 if r != 'Caucasian' else 1 for r in df.race]

d = p.get_prov_binarizer(df, ['race'])

get_prov_binarizer function took 869.079 ms


In [6]:
# Make two_year_recid the label
df = df.rename({'two_year_recid': 'label'}, axis=1)

# reverse label for consistency with function defs: 1 means no recid (good), 0 means recid (bad)
df.label = [0 if l == 1 else 1 for l in df.label]

d = p.get_prov_feature_transformation(df, ['label'])

get_prov_feature_transformation function took 953.963 ms


In [7]:
# convert jailtime to days
df['jailtime'] = (pd.to_datetime(df.c_jail_out) - pd.to_datetime(df.c_jail_in)).dt.days

#Get provenance of space transformation
d = p.get_prov_space_transformation(df, ['c_jail_out','c_jail_in'])

get_prov_space_transformation function took 1356.535 ms


In [8]:
#drop jail in and out dates
df = df.drop(['c_jail_in', 'c_jail_out'], axis=1)

d = p.get_prov_dim_reduction(df)

get_prov_dim_reduction function took 482.111 ms


In [9]:
# M: misconduct, F: felony
df.c_charge_degree = [0 if s == 'M' else 1 for s in df.c_charge_degree]

d = p.get_prov_feature_transformation(df, ['c_charge_degree'])

get_prov_feature_transformation function took 952.245 ms


In [10]:
# Uncomment bottom line to save clean csv
#df.to_csv(savepath + 'compas_onehot.csv', index=False)

#namefile = savepath + 'COMPAScleanup_prov'
#p.save_all_graph(namefile)
#Image(namefile + '.png')